In [ ]:
import numpy as np
import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

%matplotlib inline

In [ ]:
def car1_loc_data(contents):
    data_select=pd.DataFrame(columns=contents)
    for n in np.arange(10):
        print('n=',n,' of ten')
        data_raw=pd.read_csv('~/data/5-Sanandaji/case_study_dt1.csv',nrows=100000,skiprows=range(1,n*100000))
        data_raw = data_raw[contents]
        data_select=pd.concat([data_select,data_raw])
        del data_raw
    return data_select

In [ ]:
# columns to try and identify when vehicles are refueled 
cols_gas = ["source_id", "source_Vehicle_Location_Timestamp", "source_Vehicle_Speed_Unit", 
            "source_Vehicle_Speed_Value", "source_Vehicle_FuelLevel_Value", 
            "source_Vehicle_Location_Address_State"]
data_gas = car1_loc_data(cols_gas)
# data_gas = pd.read_csv('~/data/5-Sanandaji/case_study_dt1.csv',sep=',', nrows=10000)
list(data_gas)

In [ ]:
sources = pd.read_csv("~/bcdata-mojio/brian/test1_sources.txt")
sources = sources["Car IDs"].values

In [ ]:
car_number = 2
car_info = data_gas[data_gas["source_id"]==sources[car_number]]
Ncar = car_info.shape[0] # number of records for this car

In [ ]:
state = car_info["source_Vehicle_Location_Address_State"].values
print(np.unique(state))

In [ ]:
state = car_info["source_Vehicle_Speed_Unit"]
print(np.unique(state.values))

In [ ]:
time_string = car_info["source_Vehicle_Location_Timestamp"].values
time_seconds = np.zeros(Ncar)
for j in np.arange(Ncar):
    code, t = get_time(time_string[j])
    if code != 0:
        print('Error',code,' in get_time')
    time_seconds[j] = t
plt.plot(np.arange(Ncar),time_seconds)
plt.show()

In [ ]:
speed = car_info["source_Vehicle_Speed_Value"].values
plt.plot(time_seconds/24/3600,speed,'ko')
plt.show()

In [ ]:
# Routine takes time string from the data and gives back:
# 1. flag = 0: for success, 
#           1: if the input is not a string, 
#           2: if it is not long enough
# 2. seconds since February 1, 2016
#
def get_time(dxt):
    err_flag = 0
    delta_sec = 0
    
    dt0 = datetime.datetime(2016,6,9) # Measure time since February 1, 2016
    
    if not (type(dxt) is str):
        err_flag = 1
    elif len(dxt)<19:
        err_flag = 2
    else:
        year = int(dxt[0:4])
        month = int(dxt[5:7])
        day = int(dxt[8:10])
        hour = int(dxt[11:13])
        minute = int(dxt[14:16])
        second = int(dxt[17:19])
        dt1 = datetime.datetime(
            year, month, day, hour=hour, minute=minute, second=second)
        delta = dt1-dt0
        delta_sec = delta.total_seconds()
        
    return err_flag, delta_sec

In [ ]:
# Routine to get the hour minute and second and day of the week 
# adjusted from UTC to EST. Returns:
# 1. hour
# 2. minute 
# 3. second
# 4. day of the week 0-6 M-Su
# Note: no error checking, assumes past get_time() checks above 
def get_day(dxt,state):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(state)
    year = int(dxt[0:4])
    month = int(dxt[5:7])
    day = int(dxt[8:10])
    hour = int(dxt[11:13])
    minute = int(dxt[14:16])
    second = int(dxt[17:19])
    dt_utc = datetime.datetime(
        year, month, day, hour=hour, minute=minute, second=second)
    dt_utc = dt_utc.replace(tzinfo=from_zone)
    dt_local = dt_utc.astimezone(to_zone)
    return [dt_local.hour, dt_local.minute, dt_local.second, dt_local.weekday()]